In [1]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gmaps

# Google developer API key
from config import gkey
# Configure gmaps
gmaps.configure(api_key=gkey)


# File to Load
crimes_data = "chicago_crimes.csv"

# Read the Crimes Data
crimes_data_df = pd.read_csv(crimes_data)


# Display the data table for preview
crimes_data_df

,Unnamed: 0,ID,Primary Type,Location Description,Arrest,Ward,Year,Latitude,Longitude
0,0,10000092,BATTERY,STREET,False,28.0,2015,41.891399,-87.744385
1,1,10000094,OTHER OFFENSE,STREET,True,15.0,2015,41.773372,-87.665319
2,2,10000095,BATTERY,APARTMENT,False,4.0,2015,41.813861,-87.596643
3,3,10000096,BATTERY,APARTMENT,False,3.0,2015,41.800802,-87.622619
4,4,10000097,ROBBERY,SIDEWALK,False,28.0,2015,41.878065,-87.743354
5,5,10000098,BATTERY,APARTMENT,False,4.0,2015,41.805443,-87.604284
6,6,10000099,BATTERY,APARTMENT,False,17.0,2015,41.766403,-87.649296
7,7,10000100,BATTERY,APARTMENT,False,3.0,2015,41.817553,-87.619819
8,8,10000101,NARCOTICS,STREET,True,11.0,2015,41.828138,-87.672782
9,9,10000104,BATTERY,RESIDENCE PORCH/HALLWAY,False,6.0,2015,41.717455,-87.617663


In [2]:
# Combining similar offenses together
crimes_data_df["Primary Type"] = crimes_data_df["Primary Type"].replace(
   {"THEFT": "THEFT & ROBBERY", "BURGLARY": "THEFT & ROBBERY", "MOTOR VEHICLE THEFT": "THEFT & ROBBERY",
    "ROBBERY": "THEFT & ROBBERY", "BATTERY": "ASSAULT", "CRIM SEXUAL ASSAULT": "ASSAULT", "PROSTITUTION": "SEX CRIMES",
    "SEX OFFENSE": "SEX CRIMES", "CRIMINAL TRESPASS": "CRIMINAL DAMAGE", "ARSON": "CRIMINAL DAMAGE", "OTHER OFFENSE":
    "OTHER OFFENSES", "GAMBLING":"OTHER OFFENSES", "STALKING": "OTHER OFFENSES", "KIDNAPPING": "OTHER OFFENSES", "OBSCENITY": "OTHER OFFENSES",
    "INTIMIDATION": "OTHER OFFENSES", "RITUALISM": "OTHER OFFENSES", "HUMAN TRAFFICKING": "OTHER OFFENSES",
    "CONCEALED CARRY LICENSE VIOLATION": "OTHER OFFENSES", "NON-CRIMINAL (SUBJECT SPECIFIED)": "OTHER OFFENSES",
    "NON-CRIMINAL": "OTHER OFFENSES", "OFFENSE INVOLVING CHILDREN":
    "OTHER OFFENSES", "INTERFERENCE WITH PUBLIC OFFICER": "OTHER OFFENSES","PUBLIC PEACE VIOLATION": "OTHER OFFENSES", "LIQUOR LAW VIOLATION": "OTHER OFFENSES",
    "NON - CRIMINAL": "OTHER OFFENSES", "OTHER NARCOTIC VIOLATION": "NARCOTICS", "PUBLIC INDECENCY": "OTHER OFFENSES", "DOMESTIC VIOLENCE": "OTHER OFFENSES", "WEAPONS VIOLATION": "OTHER OFFENSES", "HOMICIDE": "OTHER OFFENSES"})




In [3]:
#dataframe with only robberies:
theft_crime_df = crimes_data_df.loc[crimes_data_df["Primary Type"] == "THEFT & ROBBERY", :]
theft_crime_df["Year"] = theft_crime_df["Year"].astype('str') 
crime2017_df = theft_crime_df.loc[theft_crime_df["Year"] == "2017", :]

C:\Users\mariam\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:


# Store latitude and longitude in locations
locations2017 = crime2017_df [["Latitude","Longitude"]]                             
# Drop all rows with missing informationl
locations2017 = locations2017.dropna(how='any')
locations2017 = locations2017.reset_index(drop=True)
locations2017.head()


,Latitude,Longitude
0,41.707062,-87.631483
1,41.844510,-87.705519
2,41.894476,-87.702169
3,41.895101,-87.684379
4,41.690061,-87.618201


In [5]:
locations2017.to_csv('locations_robbery_2017.csv')